## Version 1
I was able to get to +87% accuracy with version zero and get it to train on
a CPU. 

My gut feeling is that inception layers were pretty inefficient for what I
needed to do, so I'm going to try to mess with my output layers to see if I
can keep it trainable on a CPU, but get over 90% accuracy.

I'm going to stick to EfficientNetB2 as my pre-trained model and just mess
with the output layers.

Stretch goal is probably +95% accuracy. We'll call that Version 2

In [1]:
# Imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow.keras as K
import tensorflow as tf

In [2]:
# Install tensorflow v 2.9.1 so model.save() works
if ((tf.__version__) != '2.9.1'):
    %pip install tensorflow==2.9.1
print("Tensorflow Version: " + (tf.__version__))

Tensorflow Version: 2.9.1


In [3]:
# Data Preprocessing
def preprocess_data(X, Y):
    """
    X: numpy.ndarray of shape (m, 32, 32, 3) containing the CIFAR10 data
    Y: numpy.ndarray of shape (m,) containing the CIFAR10 data labels
    Returns X_p and Y_p
    """
    X_p = K.applications.efficientnet.preprocess_input(X, data_format=None)
    Y_p = K.utils.to_categorical(Y, 10)

    return X_p, Y_p

In [4]:
# Prep variables for use in model building / training
MODEL_PATH = 'cifar10_1.h5'
optimizer = K.optimizers.Adam()
init = K.initializers.he_normal()

# Load CIFAR10 dataset
(x_train, y_train), (x_valid, y_valid) = K.datasets.cifar10.load_data()

# Pre-process the data
x_train_p, y_train_p = preprocess_data(x_train, y_train)
x_valid_p, y_valid_p = preprocess_data(x_valid, y_valid)

# input tensor
inputs = K.Input(shape=(32, 32, 3))

# Resize input
resize = K.layers.Lambda(
    lambda image: K.backend.resize_images(
    image, 240/32, 240/32, data_format='channels_last')
)(inputs)

In [5]:
# Load pretrained EfficientNetB1 Base Bodel
base_model = K.applications.EfficientNetB1(
    include_top=False,
    weights='imagenet',
    input_tensor=resize,
    input_shape=(240, 240, 3)
)
base_model.trainable = False

In [9]:
# Add output layers
"""
Add output layers here, not sure what they'll be yet
"""
out = base_model.output
# Insert layers here
out = K.layers.Conv2D(filters=6,
                      kernel_size=(5, 5),
                      padding='same',
                      activation='relu',
                      kernel_initializer=init)(out)
out = K.layers.MaxPooling2D(pool_size=(2, 2),
                            strides=(2, 2))(out)
out = K.layers.Conv2D(filters=16,
                      kernel_size=(5, 5),
                      padding='same',
                      activation='relu',
                      kernel_initializer=init)(out)
# out = K.layers.MaxPooling2D(pool_size=(2, 2),
#                             strides=(2, 2))(out)
out = K.layers.Flatten()(out)
# out = K.layers.Dense(units=120,
#                      activation='relu',
#                      kernel_initializer=init)(out)
# out = K.layers.Dense(units=84,
#                      activation='relu',
#                      kernel_initializer=init)(out)
out = K.layers.Dense(units=10,
                     activation='softmax',
                     kernel_initializer=init)(out)

In [10]:
# Compile Model
model = K.models.Model(inputs=inputs, outputs=out)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
# Train Model
model.fit(x=x_train_p,
          y=y_train_p,
          batch_size=100,
          epochs=5,
          validation_data=(x_valid_p, y_valid_p))

Epoch 1/5


: 

: 

In [11]:
# Print Model Summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 lambda (Lambda)                (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['lambda[0][0]']                 
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                            

In [ ]:
# Save Model
model.save(MODEL_PATH)

In [ ]:
# Load saved model and evaluate
load_model = K.models.load_model(MODEL_PATH)
load_model.evaluate(x_valid_p, y_valid_p, verbose=1)

2023-03-29 11:39:11.620490: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-29 11:39:11.620611: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-29 11:39:11.620649: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (BensLaptop): /proc/driver/nvidia/version does not exist
2023-03-29 11:39:11.621067: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


313/313 [==============================] - 320s 1s/step - loss: 0.3731 - accuracy: 0.8770


[0.37312865257263184, 0.8769999742507935]